# GANs: Generative Adversarial Networks
Imagine the following situation: you have a counterfeitor and a cop.   
*  The counterfeitor makes a fake money sample.  Initially, the counterfeitor is very bad at this!
*  The cop examines the fake money sample, compares it with real money, and she determines that the sample is fake.
*  Based on the information from the cop, the counterfeitor improves his performance at making fake money and tries again.
*  The cop examines the new fakes, and she determines - again - that they are indeed fake.
*  The process continues, until at some point, the cop is no longer able to tell the difference between fake and real currency.   

This is the basic outline of how a GAN works.   The GAN starts out with no knowledge of a sample, but through the use of a **generator** (the counterfeitor) and a **discriminator** (the cop), can end up with incredibly realistic versions of images, music, text, etc.

GANs a represent an example of a **unsupervised learning**, in which the model learns features about a dataset without the data being labeled.

GANS are a relatively new idea in machine learning, but they are quite interesting.  Yann LeCun - one of the major figures in machine learning - has called them “The coolest idea in deep learning in the last 20 years.”   Let see if we can understand how they work.

For this workbook, I will rely heavily on the model described in Mike Bernico's book, [Deep Learning Quick Reference](https://www.amazon.com/Deep-Learning-Quick-Reference-optimizing-ebook/dp/B0791JRGPY).

# A simple GAN Model
A simple cartoon of a GAN is shown here (from [this link](https://medium.freecodecamp.org/an-intuitive-introduction-to-generative-adversarial-networks-gans-7a2264a81394)):
![gan](gan.png)

The basic function of the GAN is the following:
* The **generator** network is fed random noise, and outputs a *fake image*.
* The **discriminator** network is fed a single image, as well as a label indicating that the image is either real or fake.   It should output 1 if the image is real, and 0 if the image is fake.

# Implementation
We will design a GAN which will generate fake - but realistic - looking MNIST digits.  For the most part, we will be using techniques that you have seen before.   However, the interplay of the two networks of generator and discriminator make the successful training of a GAN very tricky.   So we will need to add some new features in order to make the training more stable.



# The Discriminator

Lets start with the discriminator.   We have made models like this before.  In fact, the basic idea here is fairly straightforward: a simple CNN which takes 28x28x1 images (like our MNIST dataset) and then produces a single output: 1 if the image is real, and 0 is the image is fake.  

The discriminator we will use is shown below.   You will notice two new features:
* A **Leaky ReLU** layer.  This replaces the ReLU activations that we have used before.  Leaky ReLU allows the pass of a small gradient signal for negative values. 
* A Batch Normalization layer.  Batch norm works by normalizing the input features of a layer to have zero mean and unit variance. Batch norm helps to deal with problems due to poor parameter initialization.

In [1]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model
from keras.optimizers import Adam
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np

def build_discriminator(img_shape):
    input = Input(img_shape)
    x = Conv2D(32, kernel_size=3, strides=2, padding="same")(input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64, kernel_size=3, strides=2, padding="same")(x)
    x = ZeroPadding2D(padding=((0, 1), (0, 1)))(x)
    x = (LeakyReLU(alpha=0.2))(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(128, kernel_size=3, strides=2, padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(256, kernel_size=3, strides=1, padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    out = Dense(1, activation='sigmoid')(x)

    model = Model(input, out)
    print("-- Discriminator -- ")
    model.summary()
    return model

Using TensorFlow backend.
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflo

# The Generator
The generator takes a random vector - in this case a vector of length 100 - and via a series of Keras layers produces an image - in our case a 28x28x1 image.   It uses the Batch Normalization layer, as well as an **UpSampling** layer.  We have previosly used UpSampling layers when we first introduced CNN autoencoders.   The UpSampling layer repeats the rows and columns of the data by size[0] and size[1] respectively.


In [2]:

def build_generator(noise_shape=(100,)):
    input = Input(noise_shape)
    x = Dense(128 * 7 * 7, activation="relu")(input)
    x = Reshape((7, 7, 128))(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(1, kernel_size=3, padding="same")(x)
    out = Activation("tanh")(x)
    model = Model(input, out)
    print("-- Generator -- ")
    model.summary()
    return model

# Get the data
The following is a helper function to get the MNIST data.   We will end up just using the training image data, and not the test data.   **We won't use the MNIST labels at all**.  The key thing we know about the MNIST training images is that they are **real**.   We will label images below, but the only labels we need are whether the images are **real** (which is only the case if they come from MNIST) or **fake** (which is only the case if the images come from our generator model above).   We don't need to know if the real image is a 0,1,..,9,  just that it is real.

In [3]:

def load_data():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)
    return X_train



# Helper function for displaying the generator images
As we train our generator, we will want to inspect the images to see how close they appear to emulating real images.   We will call the function a number of times during each epoch that we train the networks.    The code below uses 25 random vectors of length 100, feeds them into the current version of the generator, and makes 25 fake images.   It does not display them to the screen - it saves them to a directory.

In [4]:
def save_imgs(generator, epoch, batch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d_%d.png" % (epoch, batch))
    plt.close()


# Build and compile the discriminator and generator

In [5]:
discriminator = build_discriminator(img_shape=(28, 28, 1))
discriminator.compile(loss='binary_crossentropy',
                               optimizer=Adam(lr=0.0002, beta_1=0.5),
                               metrics=['accuracy'])

generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))


-- Discriminator -- 
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8,

# Training the discriminator
We have everything we need to train the discriminator: we have real images (from MNIST) and fake images (which will come from the generator).  So to train the discriminator, we simply have to feed it batches of labeled (remeber just if they are real or fake) images.   But how do we train the generator?

# Training the generator
The generator needs information to determine how poorly (or well) its fake images are.  To do this, we will use a clever trick:  we will use the output of the same discriminator above!  

To do this, we will need to make a **third** model, which below we call the **combined** model :
*  The combined model uses the generator output, and feeds it into the discriminator.  For this step, we will tell the combined model that these fake images are actually real.
*  During the **generator training process**, we will need to freeze the weights of the discriminator, so that only the weights of the generator are adjusted.   

In [6]:
z = Input(shape=(100,))
img = generator(z)
discriminator.trainable = False
real = discriminator(img)
combined = Model(z, real)
combined.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# The Training loop
The training loop below contains the logic of how we fit all 3 models (really just the two: the generator and the discriminator).  Some things to pay attention to in the code below:

1.  There is an outer loop over epochs, and an inner loop over batches.   With a batch size of 32 and the 60k MNIST images, this means we will run the inner loop 1875 times per epoch.
2.  During each batch iteration, we feed the disrimnator 16 real images and 16 fake images.  The labels for these are 1 and 0 respectively.  Notice that we randomly sample (with replacement) the real images.   So we are not using the full data each epoch!   Might be a good idea to revisit this if we were doing it for real....
3.  In the first half of this loop, we run **train_on_batch** to train the **discriminator only**.   We use train_on_batch rather than **fit** because we need to control how we feed data to the model when fitting - train_on_batch allows us to do that.
4.  **After** this, we then send 32 new **fake** images to the **combined** model.   These images are labeled as **real** (even though they are fake).   Remember that the **same** discriminator model is used as in step 3, but for this part of the loop the discriminator weights are frozen to the values that they had at step 3.  **Only the generator** weights are changed during this step.   Since we are asking the disciminator output to be 1, we are adjusting the generator weights so that the generator gets better at producing images which the discriminator will believe are real.
5.  Every 50 batches within each epoch, we generate 50 fake images, using the current version of the generator model.
6.  As the training progresses through epochs/batches, examine how the discriminatr loss and accuracy, as well as the generator loss change.  Note: train_on_batch for the discriminator returns both the loss as well as the metric (which is the accuracy in this case).


**NOTE**: Each epoch will take **alot** of time!   So you probably will want to stop it after two epoch or so.   If you want to see some cool images after 1,2,3,4 epochs, submit the script  **pbs_gan_gpu.sh** to the pbs batch system.   In about 30 minutes you will have some amazingly real digit images in the images/ directory!

In [8]:
X_train = load_data()

epochs=2        # you need about 40 epochs to get good images
batch_size=32
save_interval=1

num_examples = X_train.shape[0]
num_batches = int(num_examples / float(batch_size))
print('Number of examples: ', num_examples)
print('Number of Batches: ', num_batches)
print('Number of epochs: ', epochs)

half_batch = int(batch_size / 2)

for epoch in range(epochs + 1):
    for batch in range(num_batches):

            # noise images for the batch
        noise = np.random.normal(0, 1, (half_batch, 100))
        fake_images = generator.predict(noise)
        fake_labels = np.zeros((half_batch, 1))

            # real images for batch
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_images = X_train[idx]
        real_labels = np.ones((half_batch, 1))

            # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, 100))
            # Train the generator
        g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
        print("Epoch %d Batch %d/%d [D loss: %.2f, acc avg: %.2f%%] [D acc real: %.2f D acc fake: %.2f], [G loss: %.2f]" %
                  (epoch,batch, num_batches, d_loss[0], 100 * d_loss[1], d_loss_real[1], d_loss_fake[1],g_loss))

        if batch % 50 == 0:
            save_imgs(generator, epoch, batch)


Number of examples:  60000
Number of Batches:  1875
Number of epochs:  2


/home/llavezzo/.local/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 0 Batch 0/1875 [D loss: 1.02, acc avg: 40.62%] [D acc real: 0.81 D acc fake: 0.00], [G loss: 0.71]
Epoch 0 Batch 1/1875 [D loss: 0.58, acc avg: 71.88%] [D acc real: 1.00 D acc fake: 0.44], [G loss: 0.76]
Epoch 0 Batch 2/1875 [D loss: 0.43, acc avg: 81.25%] [D acc real: 0.94 D acc fake: 0.69], [G loss: 0.75]
Epoch 0 Batch 3/1875 [D loss: 0.22, acc avg: 93.75%] [D acc real: 0.94 D acc fake: 0.94], [G loss: 0.83]
Epoch 0 Batch 4/1875 [D loss: 0.15, acc avg: 100.00%] [D acc real: 1.00 D acc fake: 1.00], [G loss: 0.74]
Epoch 0 Batch 5/1875 [D loss: 0.14, acc avg: 100.00%] [D acc real: 1.00 D acc fake: 1.00], [G loss: 0.76]
Epoch 0 Batch 6/1875 [D loss: 0.09, acc avg: 100.00%] [D acc real: 1.00 D acc fake: 1.00], [G loss: 0.68]
Epoch 0 Batch 7/1875 [D loss: 0.12, acc avg: 100.00%] [D acc real: 1.00 D acc fake: 1.00], [G loss: 0.65]
Epoch 0 Batch 8/1875 [D loss: 0.06, acc avg: 100.00%] [D acc real: 1.00 D acc fake: 1.00], [G loss: 0.59]
Epoch 0 Batch 9/1875 [D loss: 0.15, acc avg: 100.0

Epoch 0 Batch 77/1875 [D loss: 0.04, acc avg: 100.00%] [D acc real: 1.00 D acc fake: 1.00], [G loss: 3.02]
Epoch 0 Batch 78/1875 [D loss: 0.03, acc avg: 100.00%] [D acc real: 1.00 D acc fake: 1.00], [G loss: 3.09]
Epoch 0 Batch 79/1875 [D loss: 0.04, acc avg: 100.00%] [D acc real: 1.00 D acc fake: 1.00], [G loss: 2.90]
Epoch 0 Batch 80/1875 [D loss: 0.01, acc avg: 100.00%] [D acc real: 1.00 D acc fake: 1.00], [G loss: 3.57]
Epoch 0 Batch 81/1875 [D loss: 0.11, acc avg: 96.88%] [D acc real: 1.00 D acc fake: 0.94], [G loss: 3.83]
Epoch 0 Batch 82/1875 [D loss: 0.04, acc avg: 100.00%] [D acc real: 1.00 D acc fake: 1.00], [G loss: 2.54]
Epoch 0 Batch 83/1875 [D loss: 0.10, acc avg: 96.88%] [D acc real: 1.00 D acc fake: 0.94], [G loss: 4.45]
Epoch 0 Batch 84/1875 [D loss: 0.16, acc avg: 96.88%] [D acc real: 1.00 D acc fake: 0.94], [G loss: 3.96]
Epoch 0 Batch 85/1875 [D loss: 0.11, acc avg: 96.88%] [D acc real: 0.94 D acc fake: 1.00], [G loss: 3.42]
Epoch 0 Batch 86/1875 [D loss: 0.01, acc 

Epoch 0 Batch 154/1875 [D loss: 0.66, acc avg: 75.00%] [D acc real: 0.81 D acc fake: 0.69], [G loss: 1.52]
Epoch 0 Batch 155/1875 [D loss: 0.66, acc avg: 68.75%] [D acc real: 0.81 D acc fake: 0.56], [G loss: 2.81]
Epoch 0 Batch 156/1875 [D loss: 0.85, acc avg: 50.00%] [D acc real: 0.38 D acc fake: 0.62], [G loss: 2.53]
Epoch 0 Batch 157/1875 [D loss: 0.85, acc avg: 59.38%] [D acc real: 0.81 D acc fake: 0.38], [G loss: 4.09]
Epoch 0 Batch 158/1875 [D loss: 1.25, acc avg: 46.88%] [D acc real: 0.12 D acc fake: 0.81], [G loss: 1.88]
Epoch 0 Batch 159/1875 [D loss: 1.06, acc avg: 59.38%] [D acc real: 1.00 D acc fake: 0.19], [G loss: 3.73]
Epoch 0 Batch 160/1875 [D loss: 0.80, acc avg: 62.50%] [D acc real: 0.31 D acc fake: 0.94], [G loss: 2.74]
Epoch 0 Batch 161/1875 [D loss: 0.62, acc avg: 71.88%] [D acc real: 0.75 D acc fake: 0.69], [G loss: 1.77]
Epoch 0 Batch 162/1875 [D loss: 0.77, acc avg: 59.38%] [D acc real: 0.75 D acc fake: 0.44], [G loss: 2.65]
Epoch 0 Batch 163/1875 [D loss: 0.70,

Epoch 0 Batch 231/1875 [D loss: 0.81, acc avg: 50.00%] [D acc real: 0.69 D acc fake: 0.31], [G loss: 1.59]
Epoch 0 Batch 232/1875 [D loss: 0.55, acc avg: 81.25%] [D acc real: 0.75 D acc fake: 0.88], [G loss: 1.57]
Epoch 0 Batch 233/1875 [D loss: 0.84, acc avg: 53.12%] [D acc real: 0.56 D acc fake: 0.50], [G loss: 1.83]
Epoch 0 Batch 234/1875 [D loss: 0.59, acc avg: 62.50%] [D acc real: 0.50 D acc fake: 0.75], [G loss: 1.76]
Epoch 0 Batch 235/1875 [D loss: 0.86, acc avg: 56.25%] [D acc real: 0.69 D acc fake: 0.44], [G loss: 1.85]
Epoch 0 Batch 236/1875 [D loss: 0.44, acc avg: 84.38%] [D acc real: 0.88 D acc fake: 0.81], [G loss: 2.18]
Epoch 0 Batch 237/1875 [D loss: 0.60, acc avg: 71.88%] [D acc real: 0.62 D acc fake: 0.81], [G loss: 1.33]
Epoch 0 Batch 238/1875 [D loss: 0.87, acc avg: 50.00%] [D acc real: 0.50 D acc fake: 0.50], [G loss: 1.24]
Epoch 0 Batch 239/1875 [D loss: 0.72, acc avg: 59.38%] [D acc real: 0.62 D acc fake: 0.56], [G loss: 1.45]
Epoch 0 Batch 240/1875 [D loss: 0.55,

KeyboardInterrupt: 

# A Cool Use Case for GANs



From: https://towardsdatascience.com/generative-adversarial-networks-gans-a-beginners-guide-5b38eceece24

"In addition to generating beautiful pictures, an approach for semi-supervised learning with GANs has been developed that involves the discriminator producing an additional output indicating the label of the input. This approach enables cutting edge results on datasets with very few labeled examples. On MNIST, for example, 99.1% accuracy has been achieved with only 10 labeled examples per class with a fully connected neural network — a result that’s very close to the best known results with fully supervised approaches using all 60,000 labeled examples. This is extremely promising because labeled examples can be quite expensive to obtain in practice."

(NOTE: The above medium article is no longer available.   Here is a similar article, but without the above quote:
https://skymind.ai/wiki/generative-adversarial-network-gan )

Here is an example of how this idea (of semi-supervised learning) is implemented in practice:
https://towardsdatascience.com/semi-supervised-learning-with-gans-9f3cb128c5e

# Extra Credit - 5 pts!
This could be a bit difficult:
* Run the python version of the above code.  You can find a .py file and a .sh file in this directory to start with.  The shell script can be used with **qsub** to submit the script to a CPU (there is also one for the GPU but I did not find much evidence of a speedup and given the time to actually start a GPU job the CPU one may be your best bet).
* Modify the python code to save a version of the generator model.  You will need to train the GAN for at least 15 epochs to generate realistic images.   This will take about 30 minutes using a CPU.  It would make sense to use something like the file naming procedure used for the images to name your model (meaning you could save the model every epoch, and label the filename using the current epoch number).
* Use a version of the MNIST classsifier that you made in Assignment 10.  You should have a saved model of the trained version of that classifier - copy it here.  If you can't find it, a version that I made can be found in the scratch area:

/fs/scratch/PAS1585/GAN/fully_trained_model_cnn.h5

* Load both of these models (the CNN MNIST classifier and the generator model).  Then do the follwoing:  
    * Use the GAN generator model to generate 10000 fake digits
    * Feed them into your MNIST classifier and make two plots: 
        * A histogram of which digit your fakes were classified as.  
        * The probability of your chosen digits. (You could also see if this varies by digit but that could be overkill!)

In [29]:
from keras import models, layers
import matplotlib.pyplot as plt
import numpy as np

#load generator model
generator= build_generator()
generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
n_generator = 15
generator.load_weights('generator_'+str(n_generator)+'.h5')

#load CNN MNIST classifier
cnn_network = models.Sequential()
cnn_network.add(layers.Conv2D(30,(5,5),activation='relu',input_shape=(28,28,1)))
cnn_network.add(layers.MaxPooling2D((2,2)))
cnn_network.add(layers.Conv2D(25,(5,5),activation='relu'))
cnn_network.add(layers.MaxPooling2D((2,2)))
cnn_network.add(layers.Flatten())
cnn_network.add(layers.Dense(64,activation='relu'))
cnn_network.add(layers.Dense(10,activation='softmax'))
cnn_network.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
cnn_network.load_weights('/fs/scratch/PAS1585/GAN/fully_trained_model_cnn.h5')

#use GAN generator to generate 10000 fake images
noise = np.random.normal(0, 1, (1000, 100))
fake_images = generator.predict(noise)

from collections import defaultdict
def autovivify(levels=1, final=dict):
    return (defaultdict(final) if levels < 2 else
            defaultdict(lambda: autovivify(levels-1, final)))

#use MNIST-trained CNN to evaluate the generated images
predictions = cnn_network.predict(fake_images)

#plot CNN predicted classes of generated images
probs = np.max(predictions, axis = 1)
classes = np.argmax(predictions, axis = 1)

num_bins = 10
plt.hist(classes, num_bins, facecolor='blue', alpha=0.5)
plt.show()

-- Generator -- 
Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 100)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 6272)              633472    
_________________________________________________________________
reshape_17 (Reshape)         (None, 7, 7, 128)         0         
_________________________________________________________________
batch_normalization_51 (Batc (None, 7, 7, 128)         512       
_________________________________________________________________
up_sampling2d_33 (UpSampling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 14, 14, 128)       147584    
_________________________________________________________________
activation_49 (Activation)   (None, 14, 1

In [34]:
num_bins = 10
plt.clf()
plt.hist(classes, num_bins)
plt.show()
plt.savefig('classes.png')
print(classes)

[3 2 0 7 4 6 3 7 7 6 6 7 5 5 6 7 7 0 5 5 3 2 7 2 9 6 1 5 1 5 7 3 5 0 2 3 2
 0 3 0 3 8 2 3 0 2 0 9 5 0 3 5 5 4 2 6 3 4 7 9 3 3 3 7 2 6 3 7 0 0 9 0 7 1
 1 1 0 3 5 5 1 3 2 5 2 4 4 3 1 1 5 3 3 7 3 9 7 3 7 9 9 3 3 8 5 3 0 6 7 4 7
 8 1 3 6 7 2 7 9 5 5 7 6 9 3 4 7 0 2 3 6 2 3 8 9 7 7 3 9 6 7 4 9 0 3 5 1 3
 2 7 4 7 3 2 5 0 7 4 3 5 1 7 2 1 3 5 4 6 7 1 1 0 2 2 4 3 2 5 3 1 3 8 3 7 6
 7 7 7 2 0 1 4 1 3 9 0 1 3 0 3 9 2 3 9 2 2 3 8 5 0 5 3 5 8 3 3 6 3 3 1 5 2
 9 3 3 3 6 6 3 9 0 9 1 3 7 5 2 1 3 8 9 3 2 6 2 7 3 3 0 6 2 1 1 9 6 4 5 5 0
 3 2 9 9 2 2 5 2 9 3 8 7 3 1 7 9 4 2 3 3 8 5 9 5 2 2 3 4 1 6 6 9 3 7 9 6 4
 7 7 5 4 3 1 8 5 1 3 5 0 3 2 9 1 5 5 9 4 5 6 3 0 3 3 3 9 3 2 3 3 3 8 2 1 3
 1 5 7 5 9 4 2 7 3 1 1 4 3 3 8 9 3 6 3 6 8 9 4 4 9 9 1 1 7 6 2 0 2 9 1 9 7
 9 5 1 5 4 3 9 3 9 7 5 5 3 5 9 9 7 7 5 7 3 5 3 4 2 3 3 3 1 3 4 2 9 3 7 3 3
 3 4 3 0 6 2 9 5 3 2 5 9 6 7 4 7 9 8 7 3 2 3 6 7 5 5 2 3 7 3 7 2 1 2 0 6 1
 8 7 3 3 7 6 0 5 1 3 3 5 3 1 5 2 7 6 5 7 8 3 0 2 2 3 5 8 1 4 2 5 3 2 0 1 1
 8 6 5 5 2 2 7 0 5 0 3 9 

In [38]:
print(len(probs))
plt.clf()
plt.hist(probs, num_bins)
plt.show()
plt.savefig('probs.png')

1000
